## 2021-06-10 95%信頼区間 - 2021年度心理学統計法2 第6回 04

- IV 標本から母集団を推測しよう
    - IV-7 標準偏差 (pp. 92-95)
    - コラム 4: テレビの視聴率 (pp. 104-107)
- [YouTube](https://www.youtube.com/watch?v=UfMP7OXlvYM) (32:32)

---
**データ**

- 日本人20歳女性の身長は平均: 154.9cm, 標準偏差: 6.9, n=23
    **出典** 政府統計：([e-Stat 統計で見る日本 (2017年)](https://www.e-stat.go.jp/dbview?sid=0003224177))
    サンプルが少なすぎる…
- 日本人20歳女性の身長は平均: 158.6cm, 標準偏差: 4.2, n=14
    出典：厚生労働省：[第2部 身体情況調査の結果](https://www.mhlw.go.jp/content/000711007.pdf), p. 116
    [令和元年国民健康・栄養調査報告](https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/kenkou_iryou/kenkou/eiyou/r1-houkoku_00002.html)
- 日本の17歳になる女子高生の平均身長は157.9cm (2,994校／7,755校 - 回収率: 386.% - Nは不明)
文部科学省
- 2019年度: 日本の17歳になる女子高生の平均身長は157.9cm、標準偏差 5.34
    [学校保健統計調査 平成２７年度以降 全国表 - 年齢別　都市階級別　設置者別　身長・体重の平均値及び標準偏差](https://www.e-stat.go.jp/dbview?sid=0003146500), e-Stat (女 - 2019年度 - 計 **高等学校(17歳)の項**)

以下では文部科学省の統計にしたがって平均身長157.9cm、標準偏差5.34を用いました。

In [1]:
import numpy as np
np.set_printoptions(precision=2)

import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

μ, σ = 157.9, 5.34 # 文部科学省学校保健統計調査より

# 10,000回サンプリングした結果をヒストグラムで表示してみる
試行 = np.random.normal(loc=μ, scale=σ, size=10000)
hist, edges = np.histogram(試行, bins=50)

p = figure(plot_width=400, plot_height=400)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:])

show(p)

Loading BokehJS ...

## 散布図

- $i\ (1 \le i \le 100)$番目の調査員は200人の女性の身長を計測して、その平均値を報告した。

- 散布図の各■は$i$番目の調査員が調査した200人の女性の身長の平均値を表している。

## 度数分布（ヒストグラム）

- 100人の調査員が報告した女性の身長の平均値の度数分布（ヒストグラム）

- 身長の高低に応じて20段階に振り分けたときの度数。

    このように振り分けた各段階のことを*bin*という。

In [2]:
調査対象者数, 調査員数 = 200, 100
調査 = np.array(np.random.normal(loc=μ, scale=σ, size=調査対象者数 * 調査員数)).reshape((調査対象者数, 調査員数))

標本平均たち, 標本標準偏差たち = 調査.mean(axis=0), 調査.std(axis=0, ddof=1)
print('標本標準偏差たち[:10]', 標本標準偏差たち[:10])
調査結果の平均 = 標本平均たち.mean()
標準誤差たち = 標本標準偏差たち / np.sqrt(調査対象者数)
print(f'調査結果の平均 = {調査結果の平均:.2f}')
print(標本平均たち[:10])
print(標本標準偏差たち[:10])
print(標準誤差たち[:10])

print('平均の標準偏差', 標本平均たち.std(ddof=1))
print('95%:', 調査結果の平均 - 標本平均たち.std(ddof=1) * 1.96, 調査結果の平均 + 標本平均たち.std(ddof=1) * 1.96)


標準誤差の平均 = 標準誤差たち.mean()
print(f'標準誤差: {標準誤差の平均:.2f}')

p = figure(plot_width=800, plot_height=400,
           x_range=(μ - 標準誤差の平均 * 4, μ + 標準誤差の平均 * 4), y_range=(1, 調査員数),
           title=f'{調査員数}人の調査員がそれぞれ{調査対象者数}人の女性の身長を計測した')
p.xaxis.axis_label = '調査対象者の平均身長（標本平均）'
p.yaxis.axis_label = '調査員番号'

p.rect(x=[調査結果の平均], y=[50], width=[標準誤差の平均 * 1.96 * 2], height=[100], color='red', alpha=0.2)
p.scatter(x=標本平均たち, y=np.arange(調査員数) + 1, marker='square', size=5)
p.line(x=[調査結果の平均, 調査結果の平均], y=[0, 調査員数], color='black')
p.line(x=[μ, μ], y=[0, 調査員数], color='red')

問題な調査結果たち = []
for i in range(調査員数):
    標本平均, 標準誤差 = 標本平均たち[i], 標準誤差たち[i]
    if (μ < 標本平均 - 標準誤差 * 1.96) or (標本平均 + 標準誤差 * 1.96 < μ):
        問題な調査結果たち.append([i, 標本平均, 標準誤差 * 1.96 * 2])
        p.line(x=[標本平均 - 標準誤差 * 1.96, 標本平均 + 標準誤差 * 1.96],
               y=[i+1, i+1], color='red')
問題な調査結果たち = np.array(問題な調査結果たち)
print('真の平均が標準誤差に含まれない標本')
display(問題な調査結果たち)
p.scatter(x=問題な調査結果たち[:,1], y=問題な調査結果たち[:,0]+1, size=5, marker='square', color='red')

show(p)

hist, edges = np.histogram(調査, bins=20)
p = figure(plot_width=800, plot_height=200)
p.xaxis.axis_label = '調査対象者の身長（標本の分布）'
p.yaxis.axis_label = '度数'
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:])
p.line()
show(p)

標本標準偏差たち[:10] [5.11 5.69 5.1  5.14 5.61 5.07 5.49 5.17 5.34 5.37]
調査結果の平均 = 157.91
[157.33 157.8  157.8  158.36 158.17 158.12 158.22 158.05 158.32 158.16]
[5.11 5.69 5.1  5.14 5.61 5.07 5.49 5.17 5.34 5.37]
[0.36 0.4  0.36 0.36 0.4  0.36 0.39 0.37 0.38 0.38]
平均の標準偏差 0.4243563120979022
95%: 157.0787032582952 158.74218000171896
標準誤差: 0.38
真の平均が標準誤差に含まれない標本


array([[ 21.  , 158.67,   1.4 ],
       [ 30.  , 157.08,   1.54],
       [ 37.  , 157.04,   1.51],
       [ 38.  , 156.73,   1.7 ],
       [ 43.  , 156.95,   1.5 ],
       [ 61.  , 156.92,   1.47],
       [ 65.  , 156.97,   1.35],
       [ 89.  , 158.64,   1.4 ]])